# API City prices for living cost from numbeo.com

In [ ]:
# import libraries
import requests
import os
import pandas as pd
import json
import sql_functions as sf

In [ ]:
# import load_dotenv to be able to pull the API key from the .env
from dotenv import load_dotenv
load_dotenv()

Info from API numbeo.com

yearLastUpdate': 2023

Our day_cost_bucket:
'item_id': 1, 'item_name': 'Meal, Inexpensive Restaurant, Restaurants'
'item_id': 3, 'item_name': 'McMeal at McDonalds (or Equivalent Combo Meal), Restaurants'
'item_id': 4, 'item_name': 'Domestic Beer (0.5 liter draught), Restaurants'
'item_id': 7, 'item_name': 'Water (0.33 liter bottle), Restaurants'
'item_id': 114, 'item_name': 'Cappuccino (regular), Restaurants'

'item_id': 13, 'item_name': 'Water (1.5 liter bottle), Markets'

'item_id': 18, 'item_name': 'One-way Ticket (Local Transport), Transportation'

'item_id': 108, 'item_name': 'Taxi 1km (Normal Tariff), Transportation'




Selection list of items from numbeo.com for day_cost_bucket:
'item_id': 1, 'item_name': 'Meal, Inexpensive Restaurant, Restaurants'
'item_id': 2, 'item_name': 'Meal for 2 People, Mid-range Restaurant, Three-course, Restaurants'
'item_id': 3, 'item_name': 'McMeal at McDonalds (or Equivalent Combo Meal), Restaurants'
'item_id': 4, 'item_name': 'Domestic Beer (0.5 liter draught), Restaurants'
'item_id': 6, 'item_name': 'Coke/Pepsi (0.33 liter bottle), Restaurants'
'item_id': 7, 'item_name': 'Water (0.33 liter bottle), Restaurants'
'item_id': 114, 'item_name': 'Cappuccino (regular), Restaurants'

'item_id': 13, 'item_name': 'Water (1.5 liter bottle), Markets'

'item_id': 18, 'item_name': 'One-way Ticket (Local Transport), Transportation'



'item_id': 108, 'item_name': 'Taxi 1km (Normal Tariff), Transportation'



In [ ]:
#Load city list from data/cities_numbeo excel file
city_list = pd.read_excel('data/Cities_numbeo.xlsx')


In [ ]:
city_list.head()

In [ ]:
#Split the list in three columns on comma ' , ' 
city_list[['city', 'country', 'country_2']] = city_list['City'].str.split(', ', expand=True)

In [ ]:
city_list.head()

In [ ]:
# Update the 'country' column in the city_list df with values from the 'country_2' column, but only for rows where 'country_2' is not null
city_list.loc[city_list['country_2'].notnull(), 'country'] = city_list['country_2']

In [ ]:
city_list.head()

In [ ]:
# Check for unique vvalues in country column
city_list['country'].unique()

In [ ]:
# drop the columns "city" and "country_2"
city_list.drop(['City', 'country_2'], axis = 1, inplace = True)

In [ ]:
city_list.head()

In [ ]:
# extract the 'city' column from the city_list df, convert it to list and store it in the variable 'city'
city = list(city_list['city'])

In [ ]:
city

In [ ]:
# extract the 'country' column from the city_list df, convert it to list and store it in the variable 'country'
country = list(city_list['country'])

In [ ]:
country

In [ ]:
# create a dictionary with city and country for the API query
city_count_dict = dict(zip(city, country))

In [ ]:
city_count_dict

In [270]:
# #API download from www.numbeo.com 
url = 'https://www.numbeo.com/api/city_prices'
api_key = os.getenv('numbeo_api_key') # extract the value for the api key
#country_list ' don`t need, we take the values for countries from country_list in cell above 
desired_item_ids = [1, 3, 4, 7, 114, 13, 18, 108, 26]
currency = "EUR"

data_list = []

for city in city:
    # Send the HTTP GET request for each country with currency parameter
    response = requests.get(url, params={"api_key": api_key, "query": city, "currency": currency})
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Access the "prices" list from the response
        prices = data.get("prices", [])
        
        # Access the country's land information
        city = city
        
        # Initialize a list to store the desired items for the country
        desired_items = []
        
        # Iterate over the prices and check if the item_id matches the desired values
        for price in prices:
            item_id = price.get("item_id")
            if item_id in desired_item_ids:
                desired_items.append(price)
        
        # Add the desired items, land information, and currency to the data list
        for item in desired_items:
            item["city"] = city
            item["currency"] = currency
        data_list.extend(desired_items)
    else:
        print(f"Request for {city} failed with status code:", response.status_code)

# Create a DataFrame from the data list
df = pd.DataFrame(data_list)

# Print the DataFrame
df

,data_points,item_id,lowest_price,average_price,highest_price,item_name,city,currency
0,4,1,1.607828,2.572524,3.215656,"Meal, Inexpensive Restaurant, Restaurants",W,EUR
1,4,3,3.858787,5.145049,6.431311,McMeal at McDonalds (or Equivalent Combo Meal)...,W,EUR
2,4,4,1.029010,1.157636,2.250959,"Domestic Beer (0.5 liter draught), Restaurants",W,EUR
3,4,7,0.160783,0.192939,0.321566,"Water (0.33 liter bottle) , Restaurants",W,EUR
4,4,13,0.321566,0.643131,0.964697,"Water (1.5 liter bottle), Markets",W,EUR
5,2,26,64.313111,96.469667,128.626223,"Apartment (1 bedroom) in City Centre, Rent Per...",W,EUR
6,4,108,1.286262,1.286262,1.286262,"Taxi 1km (Normal Tariff), Transportation",W,EUR
7,4,114,0.964697,0.964697,1.607828,"Cappuccino (regular), Restaurants",W,EUR
8,17,1,11.000000,15.000000,20.000000,"Meal, Inexpensive Restaurant, Restaurants",s,EUR
9,8,3,9.000000,9.900000,12.000000,McMeal at McDonalds (or Equivalent Combo Meal)...,s,EUR


In [ ]:
df.head(15)

In [ ]:
# create a df backup for emergency cases
df_backup = df

In [ ]:
#drop column data_points, item_id, lowest_price and highest_price and store the data in a "df_new"
df_new = df.drop(["data_points", 'lowest_price', 'highest_price', 'currency'], axis=1)

In [ ]:
#add the column country by going though the city_count_dict 
df_new['country'] = df_new['city'].map(city_count_dict)

In [ ]:
df_new.head(14)

In [ ]:
df_new.info()

In [ ]:
# # Display the entire df without truncation for all cells/dfs in this document
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [ ]:
# Calculate price_per_day and calc_price for rows where the item_id = 26 and store them in the particular column
df_new.loc[df_new['item_id'] == 26, 'price_per_day'] = df_new.loc[df_new['item_id'] == 26, 'average_price'] / 30
df_new.loc[df_new['item_id'] == 26, 'calc_price'] = df_new.loc[df_new['item_id'] == 26, 'price_per_day'] * 3.89

# for other rows where item_id is not 26, fill with NaN or any other default value
df_new.loc[df_new['item_id'] != 26, ['price_per_day', 'calc_price']] = None

df_new.head(20)

In [ ]:
# create a new backup
df_new_backup = df_new

In [ ]:
# update the 'average_price' column in df_new with values from the 'calc_price' column, but only for rows where 'calc_price' is not null
df_new.loc[df_new['calc_price'].notnull(), 'average_price'] = df_new['calc_price']

df_new.head(20)

In [ ]:
# create a new df named df_clean by dropping the columns "price_per_day" and "calc_price" (along the axis=1 (columns))
df_clean = df_new.drop(["price_per_day", 'calc_price'], axis=1)

df_clean.head(15)

In [ ]:
# create another backup
df_clean_backup = df_clean

In [ ]:
# Calculate the tax_max(new taxi price for 3km) for items with 'item_id' equal to 108 (current 1 km taxi price)
df_clean.loc[df_clean['item_id'] == 108, 'tax_max'] = df_clean.loc[df_clean['item_id'] == 108, 'average_price'] * 3
df_clean.head(17)

In [ ]:
# update the 'average_price' column in df_clean with values from the 'tax_max' column, but only for rows where 'tax_max' is not null
df_clean.loc[df_clean['tax_max'].notnull(), 'average_price'] = df_clean['tax_max']
df_clean.head(17)

In [ ]:
# dropping the column "tax_max" (along the axis=1 (columns))
df_clean = df_clean.drop(['tax_max'], axis=1)
df_clean.head(17)

In [ ]:
df_clean.head(17)

In [ ]:
# create another backup
df_clean_backup2 = df_clean

In [ ]:
# Update the 'item_name' column in df_clean for rows where 'item_id' is 108 - replace 'Taxi 1km (Normal Tariff), Transportation' with 'Taxi 3km (Normal Tariff), Transportation'
df_clean.loc[(df_clean['item_id'] == 108) & (df_clean['item_name'] == 'Taxi 1km (Normal Tariff), Transportation'), 'item_name'] = 'Taxi 3km (Normal Tariff), Transportation'

df_clean.head(21)

In [ ]:
from sql_functions import get_dataframe

#Get the df with the airbnb prices from sql data base
schema = 'capstone_travel_index'
airbnb_df = get_dataframe(f'SELECT * FROM {schema}.airbnb_prices_all')


In [ ]:
# create a backup for this file
airbnb_df_backup = airbnb_df

In [ ]:
# dropping the columns with the other room types
airbnb_df_clean = airbnb_df.drop(['Hotel room','Private room','Shared room'], axis=1)
airbnb_df_clean

In [ ]:
airbnb_df_clean.info()

In [ ]:
df_clean.info()

In [ ]:
# update 'average_price' in df_clean with 'Entire home/apt' value from airbnb_df_clean,
# for rows where 'item_id' is 26 and 'city' matches the current iteration city
for city in list(airbnb_df_clean['city']):
    df_clean.loc[(df_clean['item_id'] == 26) & (df_clean['city'] == city), 'average_price'] = airbnb_df_clean.query(f"city == '{city}'")['Entire home/apt'].values[0]

In [ ]:
df_clean.info()

In [ ]:
df_clean.head(17)

In [ ]:
# create another backup
df_backup5 = df_clean

In [ ]:
from country_iso_dict import country_iso_dict

# Add a new column "iso3" with ISO3 values pulled from the dict in country_iso_dict.py (mapping)
df_clean['iso3'] = df_clean['country'].map(country_iso_dict)

df_clean.head(17)

In [ ]:
# create a new df df_isnull with unique countries from df_clean where the 'iso3' column is null - countries where mapping has failed
df_isnull = df_clean[df_clean['iso3'].isnull()].country.unique()
df_isnull

In [ ]:
# array(['United Kingdom', 'Ivory Coast', 'United States',
#        'Bosnia And Herzegovina', 'Venezuela', 'Moldova', 'Vietnam',
#        'Syria', 'Tanzania', 'Isle Of Man', 'Hong Kong (China)', 'Iran',
#        'Russia', 'Taiwan', 'Macao (China)', 'Trinidad And Tobago',
#        'Kosovo (Disputed Territory)', 'Palestine', 'Bolivia',
#        'South Korea'], dtype=object)

In [ ]:
# create a new/complete dict for countries where mapping has failed
missing_countries_dict = {
    'United Kingdom': 'GBR',
    'Ivory Coast': 'CIV',
    'United States': 'USA',
    'Bosnia And Herzegovina': 'BIH',
    'Venezuela': 'VEN',
    'Moldova': 'MDA',
    'Vietnam': 'VNM',
    'Syria': 'SYR',
    'Tanzania': 'TZA',
    'Isle Of Man': 'IMN',
    'Hong Kong (China)': 'HKG',
    'Iran': 'IRN',
    'Russia': 'RUS',
    'Taiwan': 'TWN',
    'Macao (China)': 'MAC',
    'Trinidad And Tobago': 'TTO',
    'Kosovo (Disputed Territory)': 'XKX',
    'Palestine': 'PSE',
    'Bolivia': 'BOL',
    'South Korea': 'KOR'
}

In [ ]:
# create another backup
df_backup6 = df_clean

In [ ]:
# replace the NaN values in the 'iso3' column with the corresponding ISO3 values from the new dictionary
df_clean.loc[df_clean['iso3'].isnull(), 'iso3'] = df_clean.loc[df_clean['iso3'].isnull(), 'country'].map(missing_countries_dict)

In [ ]:
# check again for unique countries where the 'iso3' column is null - should be empty now
df_isnull = df_clean[df_clean['iso3'].isnull()].country.unique()
df_isnull

In [ ]:
df_clean.head(19)

In [ ]:
# create another backup
df_items_backup = df_items

In [ ]:
# create a need df with item_id and item_name columns only - plus only the values of the first 9 rows of those columns
df_items = df_clean[['item_id', 'item_name']]
df_items.drop(df_items.index[9:], axis=0, inplace=True)
df_items.head(10)

In [ ]:
# create another backup
df_backup_before_drop = df_clean

In [ ]:
# drop of the column item_name
df_clean.drop(['item_name'], axis=1, inplace=True)

In [ ]:
df_clean.head(18)

In [ ]:
# df_transposed = df_clean.pivot_table(index='city', columns=['iso3', 'country', 'item_id'], values='average_price', dropna=False).reset_index()

In [ ]:
df_clean.info()

In [ ]:
df_backup7 = df_clean

In [ ]:
# drop duplicates which have the same values in all 3 columns ('item_id', 'city', 'country')
df_clean_drop = df_clean.drop_duplicates(['item_id', 'city', 'country'])

In [ ]:
df_clean_drop.head(17)

In [ ]:
# check again for combination of the 3 columns having the same values - should be empty now
df_clean_drop[df_clean_drop.duplicated(['item_id', 'city', 'country'])]

In [ ]:
# create another backup
df_clean_drop_backup = df_clean_drop

In [ ]:
# # wrong solution with "pivot_table" instead of "pivot" 
# df_pivoted = df_clean_drop.pivot_table(index=['city', 'iso3', 'country'], columns=['item_id'], values='average_price', dropna=False).reset_index()

In [ ]:
# create a pivot table df_pivoted from the df_clean_drop df, using 'city' and 'iso3' as index,
# 'item_id' as columns, and 'average_price' as the values to populate the table
df_pivoted = df_clean_drop.pivot(index=['city', 'iso3'], columns=['item_id'], values='average_price').reset_index()

In [ ]:
df_pivoted.head(30)

In [ ]:
df_pivoted.info()

In [ ]:
# import function to check how many % of each column are NULL values
from transform_esg import per_null

In [ ]:
per_null(df_pivoted)

In [ ]:
# a last backup and renaming of the final df
df_city_prices_final = df_pivoted

In [ ]:
df_city_prices_final.head(18)

## Push the DataFrame df to sql

In [ ]:
#Define schema for DBeaver
schema = 'capstone_travel_index'
engine = sf.get_engine()

In [ ]:
#Import dataset "df_city_prices_final" to DBeaver
table_name = 'city_prices_final'
if engine!=None:
    try:
        df_city_prices_final.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

In [ ]:
# short check before upload
df_items

In [ ]:
#Import dataset "df_items" to DBeaver
table_name = 'df_items'
if engine!=None:
    try:
        df_items.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

In [ ]:
#The city_living_prices table was imported successfully.